#####
PART 2 - FORM INSTRUCTIONS: 

2.1. Make a regression model

    The model should be selected from Linear Regression or Decision Tree regression (Use scikit learn implementations!)
    Create two models:
        1) with the full dataset (direct variables)
        2) with a projection of the full data in a smaller feature space
    Discuss your results
#####

In [ ]:
''' Linear regression of the full dataset '''

#### STill working on this one. Not sure what I am doing.... ######

import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# associate data with the variables here
X = Superconduct_X ##### Replace here with full clean data
y = Superconduct_y ##### Replace here with full clean data

# Create and fit a linear regression model
model = linear_model.LinearRegression() # Create linear regression object
model.fit(Superconduct_X, Superconduct_y) # Train the model using the training sets

# Make predictions using the testing set
Superconduct_y_pred = model.predict(Superconduct_X_test) ##### Replace here with test

# Evaluate the model performance
print("Coefficients:", model.coef_) # print the coefficients
print("Intercept:", model.intercept_) # print the intercept
print("Mean squared error:", mean_squared_error(y, model.predict(X))) # print the MSE on training data
print("R2 score:", r2_score(y, model.predict(X))) # print the R2 score on training data

# Plot outputs
plt.scatter(Superconduct_X_test, Superconduct_y_test, color="black")
plt.plot(Superconduct_X_test, Superconduct_y_pred, color="blue", linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

LINEAR MODEL OF THE PCA 

By using a PCA, we can select a subset of principal components that capture most of the variation in the data, and use them as predictors in the regression model. This can help us avoid overfitting, multicollinearity, and noise issues that may arise from using too many or irrelevant features. Below is the linear regression of the projected data:

In [ ]:
''' Linear regression of the PCA projection '''

import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# associate data with the variables here
X = Superconduct_X_train ##### Replace here with train
y = Superconduct_y_train ##### Replace here with train

# Create and fit a linear regression model
model = linear_model.LinearRegression() # Create linear regression object
model.fit(Superconduct_X_train, Superconduct_y_train) # Train the model using the training sets

# Make predictions using the testing set
Superconduct_y_pred = model.predict(Superconduct_X_test) ##### Replace here with test

# Evaluate the model performance
print("Coefficients:", model.coef_) # print the coefficients
print("Intercept:", model.intercept_) # print the intercept
print("Mean squared error:", mean_squared_error(y, model.predict(X))) # print the MSE on training data
print("R2 score:", r2_score(y, model.predict(X))) # print the R2 score on training data

# Plot outputs
plt.scatter(Superconduct_X_test, Superconduct_y_test, color="black")
plt.plot(Superconduct_X_test, Superconduct_y_pred, color="blue", linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

Decision Tree Regression

Decision tree regression is a method of fitting a piecewise constant function to the data by learning simple decision rules inferred from the features. A decision tree is a tree-like structure that splits the data into subsets based on the values of the features. Each node in the tree represents a feature, each branch represents a decision rule, and each leaf represents an output value. The output value of a decision tree regression is the average of the target values in the leaf node2. Decision tree regression can handle both numerical and categorical features, and can capture complex nonlinear relationships in the data3.

In [ ]:
''' Decision Tree Regression '''

#### AGAIN: STill working on this one. Not sure what I am doing.... Not even sure we need the decision tree regression ######


# Import the necessary modules and libraries
import numpy as np
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt

# import dataset


# Fit regression model
regr_1 = DecisionTreeRegressor(max_depth=2 ) ### dont know what depth means
regr_2 = DecisionTreeRegressor(max_depth=5) ### dont know what deapth means
regr_1.fit(X, y)
regr_2.fit(X, y)

# Predict
X_test = np.arange(0.0, 5.0, 0.01)[:, np.newaxis]
y_1 = regr_1.predict(X_test)
y_2 = regr_2.predict(X_test)

# Plot the results
plt.figure()
plt.scatter(X, y, s=20, edgecolor="black", c="darkorange", label="data")
plt.plot(X_test, y_1, color="cornflowerblue", label="max_depth=2", linewidth=2)
plt.plot(X_test, y_2, color="yellowgreen", label="max_depth=5", linewidth=2)
plt.xlabel("data")
plt.ylabel("target")
plt.title("Decision Tree Regression")
plt.legend()
plt.show()

DISCUSSION OF PART 2.1:

bla bla, I wish I knew